## Owen Dewing - AI Makerspace Certification Challenge


### Dependencies & API Keys

In [1]:
import os 
from getpass import getpass
os.environ['OPENAI_API_KEY'] = getpass("Enter your OpenAI API key: ")

In [2]:
os.environ['TAVILY_API_KEY'] = getpass("Enter your Tavily API key: ")

### Data Preparation

In [3]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader

path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_documents = text_splitter.split_documents(docs)
len(split_documents)


1885

In [5]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

client = QdrantClient(":memory:")

client.create_collection(
    collection_name="Student_Loan_Data",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

vectorstore = QdrantVectorStore(
    client=client,
    collection_name="Student_Loan_Data",
    embedding=embeddings,
)

vectorstore.add_documents(split_documents)

['2c71805517de4bb2819f61547fbe546e',
 'd981dbb0da724432b3c6e90f0a869d83',
 '4ab726b0e4ca42028949e97e1b75b4f1',
 '2d57238da9b4416a9f1cbe24ebbf16dd',
 'dcba0e6e2d364ed18032555dd41169fa',
 '53969b6fb1224e16b5aff015b3cd6188',
 'd0725396fa12453a86288bc1c5d2e56e',
 'e47ec23a0d3142fd864168fdcccd274e',
 '411d0f9fe13d48b0a8125830d751e218',
 'ada6eca0d0b0402a9a36a19da9e689ed',
 '952731ad6d7948a49f74e56cbe7b7b5a',
 '2db97d34286043d3b110121c41d86a46',
 '18d599904f454d2e963ead38523358ee',
 '13bbe3e74e4f46d9bec66e61e0c47d6d',
 '6e6918d83ff9416283c21fc608b6e4e4',
 'e89c731af3494be8b83c2427fbd920f7',
 'e9f6e1e7f7814d2faa47e704c22e2603',
 '1a950de4a6ec445f8fb1d6cf5474e3ad',
 'a079602790ad42d6bf1ad24011a34594',
 '7c8df8a8175044c29d7f6acdd3cf3c31',
 'c90c160277d046a0b057919a8e31639c',
 '585d61eb94954513b1611e009160d0da',
 'a9c7a459754c47d7a2468c775a8ec5dd',
 '848b35d368fd4df6b2ecc4184b2b39b1',
 'a8334cf7be16452e9ccc8f983167ce14',
 '3e3c589d9fa340d39490c684ad81fb30',
 '8149008e427d47b5b52758549fd1c602',
 

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k" : 10})

In [8]:
def retrieve(state):
  retrieved_docs = retriever.invoke(state["question"])
  return {"context" : retrieved_docs}

In [9]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
You are a helpful assistant who answers questions based on provided context. You must only use the provided context to answer the question. If you do not know the answer, or it's not contained in the provided context response with "I don't know"

Context:
{context}

Question:
{question}

Answer:
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [10]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-4.1-nano")

In [11]:
def generate(state):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = rag_prompt.format_messages(question=state["question"], context=docs_content)
  response = openai_chat_model.invoke(messages)
  return {"response" : response.content}

In [12]:
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  response: str

In [13]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [14]:
response = graph.invoke({"question" : "What is the Repayment Assistance Plan, and when will it be introduced??"})

In [15]:
response["response"]


'The Repayment Assistance Plan is an income-based repayment plan that the Secretary shall carry out beginning on July 1, 2026. It has terms and conditions including that the total monthly repayment amount for all loans repaid under this plan will be equal to a calculated applicable monthly payment, subject to certain limits, and that the Secretary will apply this payment first toward the borrower’s eligible loans.'

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=5)
#This tool will get the latest news/updates on the bill and the student loan repayment plan

/var/folders/qn/dpq4ksjx4h96cz7_nbv5sgrh0000gn/T/ipykernel_13765/2191912045.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


In [17]:
from backend.main import timeline_tool, complete_form_tool, tool_comparison_tool

tool_belt = [
    tavily_tool,
    timeline_tool,
    complete_form_tool,
    tool_comparison_tool,
]

In [18]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1", temperature=0)

In [19]:
model = model.bind_tools(tool_belt)

In [22]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.documents import Document

class AgentState(TypedDict):
    messages: Annotated[list, add_messages]
    context: List[Document]

In [24]:
from langgraph.prebuilt import ToolNode

def call_model(state):
    messages = state["messages"]
    response = model.invoke(messages)
    return {
        "messages" : [response],
        "context" : state.get("context", [])
        }


tool_node = ToolNode(tool_belt)

In [25]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

In [26]:
uncompiled_graph.set_entry_point("agent")

In [27]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

In [28]:
uncompiled_graph.add_edge("action", "agent")

In [29]:
compiled_graph = uncompiled_graph.compile()

In [30]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_INezRtdYF6Jn7Zpj9lnv2jsq', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 226, 'total_tokens': 249, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-C158rZZ7TcevvVAncX6SNF8EHSgra', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b422f11b-09f0-4eb4-b070-f13221bff581-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_INezRtdYF6Jn7Zpj9lnv2jsq', 't

In [31]:
inputs = {"messages": [HumanMessage(content = "Has the new megabill RAP plan made it harder for students to pay off loans?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node =="action":
            print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_QC52Tjk2OkfRgkzZhTwnGb4Q', 'function': {'arguments': '{"query":"impact of new megabill RAP plan on student loan repayment difficulty"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 234, 'total_tokens': 264, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-2025-04-14', 'system_fingerprint': 'fp_51e1070cf2', 'id': 'chatcmpl-C1592qb4QzOqi51W2WPa02mhkpun0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--08b876fa-d837-4ae0-8c5d-5e65bd48f01c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'impact of new megabill RAP plan on student loan r

In [34]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-3.5-turbo"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [35]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=5)

Applying HeadlinesExtractor:   0%|          | 0/452 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/480 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/868 [00:00<?, ?it/s]

Property 'summary' already exists in node '180073'. Skipping!
Property 'summary' already exists in node 'ec4bc7'. Skipping!
Property 'summary' already exists in node '1e4945'. Skipping!
Property 'summary' already exists in node 'eda34f'. Skipping!
Property 'summary' already exists in node '2a5eb5'. Skipping!
Property 'summary' already exists in node 'cd0605'. Skipping!
Property 'summary' already exists in node '514bb1'. Skipping!
Property 'summary' already exists in node '684b72'. Skipping!
Property 'summary' already exists in node '78a7a1'. Skipping!
Property 'summary' already exists in node '143c44'. Skipping!
Property 'summary' already exists in node '1ed57f'. Skipping!
Property 'summary' already exists in node 'a40761'. Skipping!
Property 'summary' already exists in node 'eaf8f7'. Skipping!
Property 'summary' already exists in node 'bd584c'. Skipping!
Property 'summary' already exists in node '50e7af'. Skipping!
Property 'summary' already exists in node '61d048'. Skipping!
Property

...

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

unable to apply transformation: Node 05582bb4-0fcb-4fbe-b082-495e168f8053 has no summary_embedding


Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/2 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/6 [00:00<?, ?it/s]

In [36]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What are the restrictions and eligibility crit...,"[information, see the discussion under <Direct...",A parent borrower must meet the same citizensh...,single_hop_specifc_query_synthesizer
1,What are the requirements for a dependent stud...,"[student. For example, if a student9s biologic...",The dependent student on whose behalf a parent...,single_hop_specifc_query_synthesizer
2,How can a Direct PLUS Loan applicant with an a...,[There are two means by which a Direct PLUS Lo...,An applicant with an adverse credit history ma...,single_hop_specifc_query_synthesizer
3,What are the modifications proposed for cost-s...,[<1-hop>\n\nPage 47 of 48 Sec. 71125. Modifyin...,The modifications proposed for cost-sharing re...,multi_hop_specific_query_synthesizer
4,What are the exceptions to the normal loan per...,[<1-hop>\n\nMinimum Loan Periods The minimum p...,The exceptions to the normal loan period and d...,multi_hop_specific_query_synthesizer
5,How does Chapter 5 impact tax deductions and c...,[<1-hop>\n\nMinimum Loan Periods The minimum p...,Chapter 5 impacts tax deductions and credits f...,multi_hop_specific_query_synthesizer


In [37]:
for test_row in dataset:
  response = graph.invoke({"question" : test_row.eval_sample.user_input})
  test_row.eval_sample.response = response["response"]
  test_row.eval_sample.retrieved_contexts = [context.page_content for context in response["context"]]


In [38]:
dataset.to_pandas()

,user_input,retrieved_contexts,reference_contexts,response,reference,synthesizer_name
0,What are the restrictions and eligibility crit...,[students.\nA parent may receive a Direct PLUS...,"[information, see the discussion under <Direct...",The restrictions and eligibility criteria for ...,A parent borrower must meet the same citizensh...,single_hop_specifc_query_synthesizer
1,What are the requirements for a dependent stud...,"[student. For example, if a student9s biologic...","[student. For example, if a student9s biologic...",The requirement for a dependent student to fil...,The dependent student on whose behalf a parent...,single_hop_specifc_query_synthesizer
2,How can a Direct PLUS Loan applicant with an a...,[An applicant cannot be denied a Direct PLUS L...,[There are two means by which a Direct PLUS Lo...,A Direct PLUS Loan applicant with an adverse c...,An applicant with an adverse credit history ma...,single_hop_specifc_query_synthesizer
3,What are the modifications proposed for cost-s...,[Page 47 of 48 \n \nSec. 71125. Modifying cost...,[<1-hop>\n\nPage 47 of 48 Sec. 71125. Modifyin...,The modifications proposed for cost-sharing re...,The modifications proposed for cost-sharing re...,multi_hop_specific_query_synthesizer
4,What are the exceptions to the normal loan per...,[Work in a Standard Term=). If a standard term...,[<1-hop>\n\nMinimum Loan Periods The minimum p...,I don't know,The exceptions to the normal loan period and d...,multi_hop_specific_query_synthesizer
5,How does Chapter 5 impact tax deductions and c...,"[Sec. 70524. Income from hydrogen storage, car...",[<1-hop>\n\nMinimum Loan Periods The minimum p...,I don't know,Chapter 5 impacts tax deductions and credits f...,multi_hop_specific_query_synthesizer


In [39]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

In [40]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-3.5-turbo"))

In [41]:
from ragas.metrics import Faithfulness, ResponseRelevancy, ContextEntityRecall, ContextPrecision
from ragas import evaluate, RunConfig

custom_run_config = RunConfig(timeout=500)

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[ContextPrecision(), Faithfulness(), ResponseRelevancy(), ContextEntityRecall()],
    llm=evaluator_llm,
    run_config=custom_run_config
)
result

Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]

Exception raised in Job[11]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)
Exception raised in Job[15]: LLMDidNotFinishException(The LLM generation was not completed. Please increase the max_tokens and try again.)


{'context_precision': 1.0000, 'faithfulness': 0.6500, 'answer_relevancy': 0.6573, 'context_entity_recall': 0.2110}